# Исследование надежности заемщиков

**Цель исследования** — разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. А также влияет ли уровень дохода и цели кредита на возвращения кредита в срок.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**План исследования**

1. Сделать обзор данных 
2. Предобработка данных
3. Анализ данных и ответы на поставленные вопросы

**Согласно документации к данным:**
* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

# Изучение данных

Изучим данные, предоставленные сервисом для проекта.

## Импорт библиотек

In [1]:
import os

import pandas as pd # импорт библиотеки pandas

## Функции

In [2]:
def categorize_income(income):
    
    '''
    Функция определяет категорию клиента
    в зависимости от его дохода
    '''
    
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [3]:
def categorize_purpose(row):
    
    '''
    Функция определяет категорию для
    цели получения кредита
    '''
    
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [4]:
def que(category):
    
    '''
    Функция создает сводную таблицу для
    определения общего количества выданных кредитов,
    всех должников и долю должников от общего количества
    
    '''
    
    data_temp = temp.pivot_table(index=category, values='debt', aggfunc=['count', 'sum', 'mean'])
    data_temp.columns = ['Всего кредитополучателей', 'Всего должников', 'Доля должников']
    # Оформим таблицу цветным градиентом
    display(data_temp.style.format("{:.3f}").background_gradient(cmap='Blues', axis=0))

## Загрузка данных

In [5]:
pth = r"C:\Users\stepa\Downloads\data.csv"

In [6]:
if os.path.exists(pth):
    data = pd.read_csv(pth)
else:
    print('Ошибка')

Получение первых 10 строк таблицы.

In [7]:
data.head(10) # получение первых 10 строк таблицы data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Общая информация о данных таблицы `data`

In [8]:
data.info() # получение общей информации о данных в таблице data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Вывод

Каждая сторка таблиц содержит информацию о заёмщиках. Необходимо сделать проверку данных:

- Cделать проверку на пропуски;
- Сделать проверку на явные и неявные дубликаты;
- Изменить тип данных в столбце `total_income` на `int`;
- Сделать новый столбце с категориями на основании доходов клиентов.

# Предобработка данных

## Работа с пропусками

Выведим количество пропущенных значений для каждого столбца. 

In [9]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Это `days_employed`и `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполним пропуски в этом столбце медианным значением по каждому типу из столбца `income_type`.

In [10]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

## Обработка аномальных значений

В столбце `days_employed` встречаются артефакты (аномалии). В этом столбце есть отрицательное количество дней трудового стажа. Обработаем значения в данном столбце.

In [11]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведим медианное значение трудового стажа `days_employed` в днях.

In [12]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, но этот столбец не понадобится нам для исследования, поэтому оставим как есть.

Выведим перечень уникальных значений столбца `children`.

In [13]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.

In [14]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Ещё раз выведим перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.

In [15]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

## Удаление пропусков (продолжение)

Заполним пропуски в столбце `days_employed` медианными значениями по каждому типу занятости `income_type`.

In [16]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски заполнены. Ещё раз выведим количество пропущенных значений для каждого столбца.

In [17]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

## Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный.

In [18]:
data['total_income'] = data['total_income'].astype(int)

## Работа с дубликатами

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру.

In [19]:
data['education'] = data['education'].str.lower()

Выведим на экран количество строк-дубликатов в данных. И если такие строки присутствуют, удалим их.

In [20]:
data.duplicated().sum()

71

In [21]:
data = data.drop_duplicates()

## Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями, для этого была написана функция `categorize_income`:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [22]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведим на экран перечень уникальных целей взятия кредита из столбца `purpose`

In [23]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию `categorize_purpose`, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.


In [24]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Вывод

На данном этапе были сделаны следующие действия:
1. Сделали проверку на пропуски:
    - В столбце `total_income` заполнили пропуски в медианным значением по каждому типу из столбца `income_type`.
    - В столбце `days_employed` заполнили пропуски медианными значениями по каждому типу занятости `income_type`.
2. Обработали аномальные значения:
    - В столбце `days_employed` избавились от отрицательных количеств дней трудового стажа.
    - В столбце `children` удалили аномальные значения.
3. Изменили тип данных в столбце `total_income` на `int`.
4. Сделали проверку на неявные дубликаты:
    - В столбце `education` привели значения к нижнему регистру.
5. Сделали проверку на янвые дублика - **удалили их**
6. Добавили новый столбец `total_income_category` с категориями клиентов в зависимости от их дохода
7. Добавили новый столбец `purpose_category` с категориями для целей кредита.

# Исследование данных и ответы на вопросы
## Есть ли зависимость между количеством детей и возвратом кредита в срок?

Для того, чтобы выяснить это, напишем функцию `que`, в которой сделаем группировку по количеству детей. А также так, как в столбце `debt` значения могут принимать только `0` и `1`, то можно найти количество должников в зависимости от количества детей, а еще найдем количество людей в каждой группе. Для этого воспользуемся методом `aggfunc` с фнуккциями `sum`, `count` и `mean`.

In [25]:
# Создаем копию датасета, что бы не работать с исходным
temp = data.copy()

In [26]:
que('children')

,Всего кредитополучателей,Всего должников,Доля должников
children,,,
0,14091.000,1063.000,0.075
1,4808.000,444.000,0.092
2,2052.000,194.000,0.095
3,330.000,27.000,0.082
4,41.000,4.000,0.098
5,9.000,0.000,0.000


### Вывод. 

Принимая во внимание то, что у нас не равные выборки, а также исключим категорию с 5-тью детьми. Можно разбить людей на две категории:
* Люди, которые не имеют детей
* Люди с детьми 

Исходя из этих двух групп делаем вывод, что первая группа (люди без детей) чаще возвращают кредиты в срок.

## Есть ли зависимость между семейным положением и возвратом кредита в срок?

Мы заранее написали функцию, так как задачи анологичные, поэтому воспользуемся написанной ранее функцией для стобца `family_status`.

In [27]:
que('family_status')

,Всего кредитополучателей,Всего должников,Доля должников
family_status,,,
Не женат / не замужем,2796.000,273.000,0.098
в разводе,1189.000,84.000,0.071
вдовец / вдова,951.000,63.000,0.066
гражданский брак,4134.000,385.000,0.093
женат / замужем,12261.000,927.000,0.076


### Вывод.

По полученным данным можно сгруппировать людей по двум группам:

1. Люди, которые никогда не состояли в официальном браке (гражданский или не женаты / не замужем)
2. Люди, которые состояли в официальном браке

Исходя из данной группировки можно сказать, что люди, относящиеся ко второй группе, чаще возвращают кредиты в срок.

## Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Анологичная задача, поэтому также воспользуемся функцией для столбца `total_income_category`.

In [28]:
que('total_income_category')

,Всего кредитополучателей,Всего должников,Доля должников
total_income_category,,,
A,25.000,2.000,0.080
B,5014.000,354.000,0.071
C,15921.000,1353.000,0.085
D,349.000,21.000,0.060
E,22.000,2.000,0.091


### Вывод.

Из-за того, что выборки не равны и как можно увидеть, что в 3-ёх категориях недостаточный размер выборки для вывода, поэтому придется их не учитывать. В результате, возьмем оставшиеся две группы `B` и `C`.

В итоге прослеживается следующая зависимость, что чем больше у человека доход, тем чаще он возвращает кредиты в срок.

## Как разные цели кредита влияют на его возврат в срок?

Сделаем анологичные действия, но только для столбца `purpose_category`.

In [29]:
que('purpose_category')

,Всего кредитополучателей,Всего должников,Доля должников
purpose_category,,,
операции с автомобилем,4279.000,400.000,0.093
операции с недвижимостью,10751.000,780.000,0.073
получение образования,3988.000,369.000,0.093
проведение свадьбы,2313.000,183.000,0.079


### Вывод.

По полученным данным можно сформировать две группы:

1. Люди, которые взяли кредит для недвижимости или свадьбы.
2. Люди, которые взяли кредит для образования или автомобиля.

Можно сказать, что первая группа людей чаще возваращает кредиты в срок, чем вторая группа.

# Общий вывод.

Данные были получены из файла `data.csv`. О качестве данных ничего не известно, поэтому сначала сделали обзор данных. Само исследование проходило в три этапа:
1. Обзор данных
2. Предобработка данных
3. Анализ и ответы на поставленные вопросы

**1. Обзор данных.** 

В таблице 12 столбцов. Тип данных в столбцах — `object`(5), `float`(2) и `int`(5)

Согласно документации к данным:
* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

**Вывод**

Количество значений в двух столбцах различается. Значит, в данных есть пропущенные значения.

**2. Предобработка данных.**

Пропуски данных были в двух столбцах `total_income` и `days_employed`. 

* В начале заполнили пропуски в `total_income` медианным значением по каждому типу из столбца `income_type`, так как тип занятости влияет на уровень дохода.
* Следующая аномалия заключалась в том, что в столбце `days_employed` встречались отрицательные значения - заменили их на положительные.
* Для каждого типа занятости было получено медианное значение по столбцу `days_employed` и в итоге были выявлены аномальные значения для безработных и пенсионеров, но на данное исследование они не повлияют.
* Удалили строки с аномальными значениями в столбце `children`. 
* Заполнили  пропуски в столбце `days_employed` медианными значениями по каждому типу занятости `income_type`.
* Обработали неявные дубликаты в столбце `education` встречались одни и те же значения, но написанные в разном регистре - привели к одному регистру.
* Назначали категорию каждому человеку в зависимости от его дохода от `Е` до `А`, где `Е` - самый минимальный доход.
* Назначали категорию людям в зависимости от их целей, то есть операции с недвижимостью или автомобилями, получение образования и проведение свадьбы.

**3. Анализ и ответы на поставленные вопросы**

Необходимо было ответить на четыре вопроса:

1. Есть зависимость между количеством детей и возвратом кредита в срок?

В ходе данного исследование пришлось принимать во внимание то, что выборки получились разные. Также из-за сильного выброса пришлось исключить категорию людей с пятью детьми, так как данная категория в данном случае является нерелевантной. Принимая во внимание все исключения в итоге разбили людей на две категории, первая из которых чаще возвращает кредиты в срок:

* Люди, которые не имеют детей
* Люди с детьми

Что в результате дает нам ответ на поставленный вопрос, что зависимость между количеством детей и возвратом кредита в срок **есть**.

2. Есть ли зависимость между семейным положением и возвратом кредита в срок? Ответ: **есть**

В ходе данного исследование было установлено, что люди, которые никогда не состояли в официальном браке, хуже возвращают кредиты в срок. В результате было выделено две группы людей:

* Люди, которые никогда не состояли в официальном браке
* Люди, которые состояли в официальном браке

3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок? Ответ: **частично есть**

На этом этапе пришлось исключить три категории из пяти. Дело все в том, что в этих категориях недостаточный объем выборки.
В итоге прослеживается следующая зависимость, что чем больше у человека доход, тем чаще он возвращает кредиты в срок, но надо учитывать тот факт, что исследование было неполным из-за исключения трех категорий.

4. Как разные цели кредита влияют на его возврат в срок?

В результате данного исследования можно разделить людей на две группы, в первую входят те, кто чаще закрывает кредиты в срок:

1. Люди, которые взяли кредит для недвижимости или свадьбы.
2. Люди, которые взяли кредит для образования или автомобиля.

**Рекомендации.**

Идеальный заемщик может быть следующим:

* У него нету детей
* Когда-либо состоял в официальном браке
* Доход от 200000
* Берет кредит на недвижимость или для проведения свадьбы